In [2]:
!pip install -U stable-ts
!pip3 install pyTelegramBotAPI
!pip3 install --upgrade imageio-ffmpeg
!pip install moviepy==1.0.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 74.7 MB/s eta 0:00:00
  Created wheel for stable-ts: filename=stable_ts-1.4.0-py3-none-any.whl size=38882 sha256=3db42988ef2c9184efb263c50b88f36f684b5d6a06623e75ee15b1dbda67f1d6
  Stored in directory: /root/.cache/pip/wheels/31/f1/bc/9890263c3f024895f45f25a479f977ef0bab8267ce08a92adf
  Created wheel for openai-whisper: filename=openai_whisper-20230124-py3-none-any.whl size=1179328 sha256=aa24b75cdf3d18bac4e232eb95d058139dc6647e94c832302ab77b969738b914
  Stored in directory: /root/.cache/pip/wheels/c2/0f/ce/61fc86ca933

In [ ]:
# This is a sample Python script.
import os
import telebot
from moviepy.audio.AudioClip import CompositeAudioClip
from moviepy.audio.io.AudioFileClip import AudioFileClip
from moviepy.video.io.VideoFileClip import VideoFileClip
import moviepy.audio.fx.all as afx
import whisper 
from stable_whisper import modify_model
from stable_whisper.text_output import finalize_segment_word_ts, to_srt
import random
# root_path = '/home/nikita/Sound/bot'

def prepare_storage_path(raw_path: str):
    path = raw_path[raw_path.find('/'):]
    print(path)
    return './storage' + path

model = whisper.load_model('small')
modify_model(model)


    



100%|███████████████████████████████████████| 461M/461M [00:07<00:00, 63.2MiB/s]


In [ ]:
def bot_init():
    # создаем объект бота
    bot = telebot.TeleBot("6285769988:AAHOCxWhVLL-ELy1pKQhUwDOD37bC27a1QQ")
    os.makedirs(os.path.dirname('storage/'), exist_ok=True)

    @bot.message_handler(content_types=['text'])
    def chat(message):
        if message.text == 'Привет':
            bot.send_message(message.from_user.id,
                             'Отправь аудио, видео или голосовое боту. Он почистит твой голос от мата и паразитных слов.')

    @bot.message_handler(content_types=['audio', 'voice', 'video'])
    def file_process(message):
        # receive
        if message.content_type == 'audio':
            file_info = bot.get_file(message.audio.file_id)
        if message.content_type == 'voice':
            file_info = bot.get_file(message.voice.file_id)
        if message.content_type == 'video':
            file_info = bot.get_file(message.video.file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        storage_file_path = prepare_storage_path(file_info.file_path)
        with open(storage_file_path, 'wb') as new_file:
            new_file.write(downloaded_file)
        print('Download completed')
        bot.send_message(message.from_user.id, 'Аудио получено. Подожди немного...')

        if message.content_type == 'video':
            storage_video_file_path = storage_file_path
            audio_file_path = convert_video_to_audio_moviepy(storage_file_path)

        clip = VideoFileClip(storage_video_file_path)
        audio_clip = AudioFileClip(audio_file_path)
        # audio_beep = AudioFileClip("./media/beep.mp3").set_duration(1)
        

        result = model.transcribe(audio_file_path, language='ru', suppress_silence=True, ts_num=16, lower_quantile=0.05, lower_threshold=0.1)
        print(result)
        segs = finalize_segment_word_ts(result, combine_compound=True)
        print(segs)
        ban_words = ['еб','сука','сук',"блять","бля","пидор","гей", "еблан", "долбаёб","далбаёб","долбаеб","далбаеб","жопа","зад","дроч","ху","ублюдок","пиздец","пизда","пиз","еба"]

        to_ban = []
        for i in segs:
          for c in range(len(i[0])):
            if i[0][c].lower() in ban_words:
              to_ban.append((i[0][c],i[1][c]))
            else:
              for ban in ban_words:
                if ban in i[0][c].lower():
                  to_ban.append((i[0][c],i[1][c]))
                  break
        print(to_ban)
        picks = [(x[1]['start']-0.1,x[1]['end']+0.1) for x in to_ban]
        # -----------------
        # pics audio
        arr = []
        last_end = 0
        for start_pick, end_pick in picks:
           audio_beep = AudioFileClip("./media/qqq.m4a").subclip(t_start = random.randint(0,24))
           arr.extend([audio_clip.subclip(t_start = last_end, t_end=start_pick).set_start(last_end), 
                                             audio_beep.set_start(start_pick).set_duration(end_pick-start_pick)])
           last_end = end_pick
        arr.append(audio_clip.subclip(t_start = last_end).set_start(last_end))
        audio_clip_composed = CompositeAudioClip(arr)
        if message.content_type == 'video':
            clip.set_audio(audio_clip_composed).write_videofile(
        "edited.mp4")

        # send
        with open("edited.mp4", 'rb') as file:
            if message.content_type == 'audio':
                bot.send_audio(message.from_user.id, audio=file)
            if message.content_type == 'voice':
                bot.send_voice(message.from_user.id, voice=file)
            if message.content_type == 'video':
                bot.send_video(message.from_user.id, video=file)

        print('Send completed')
    bot.polling()

# Press Shift+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.


def print_hi(name):
    # Use a breakpoint in the code line below to debug your script.
    print(f'Hi, {name}')  # Press Ctrl+F8 to toggle the breakpoint.





def convert_video_to_audio_moviepy(video_file, output_ext="mp3"):
    """Converts video to audio using MoviePy library
    that uses `ffmpeg` under the hood"""
    filename, ext = os.path.splitext(video_file)
    clip = VideoFileClip(video_file).fx(afx.audio_normalize)
    clip.audio.write_audiofile(f"{filename}.{output_ext}")
    return f"{filename}.{output_ext}"

In [ ]:
if __name__ == '__main__':
    
    print_hi('PyCharm')
    bot_init()


In [ ]:
 # result = model.transcribe(storage_file_path, language='ru', suppress_silence=True, ts_num=16, lower_quantile=0.05, lower_threshold=0.1)

        # segs = finalize_segment_word_ts(result)#, combine_compound=True)
        # ban_words = ['сука','сук',"блять","бля","пидор","гей", "еблан", "долбаёб","далбаёб","долбаеб","далбаеб","жопа","зад","дроч","ху","ублюдок","пиздец","пизда","пиз","еба"]

        # to_ban = []
        # for i in segs:
        #   for c in range(len(i[0])):
        #     if i[0][c] in ban_words:
        #       to_ban.append((i[0][c],i[1][c]))
        #     else:
        #       for ban in ban_words:
        #         if ban in i[0][c]:
        #           to_ban.append((i[0][c],i[1][c]))
        #           break
        # print(to_ban)